# Sprawdzenie kernela 

Na początek sprawdź czy silnik wykonawczy Twojego notatnika to PySpark. 
Mógłby on być po prostu interpreterem Pythona, jednak wówczas zmienne kontekstu musielibyśmy tworzyć samodzielnie.

Sprawdź, czy obiekt kontekstu jest dostępny. W przypadku *Spark SQL* jest to `SparkSession`

In [1]:
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import sys
import os

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

conf = SparkConf().setAppName("spark-dataframes-api-zadania")
sc = SparkContext(conf=conf)
spark = SparkSession.builder.appName("dataframes-zadania").getOrCreate()

Dzięki powyższej informacji dowiedzieliśmy się nie tylko w jakim trybie został uruchomiony Spark obsługujący nasze polecenia, w jakiej jest wersji, ale także czy obsługuje funkcjonalność platformy Hive.

Dowiedz się także pod jakim użytkownikiem działamy w ramach tego notatnika.

In [ ]:
%%sh 
whoami

Czas na nasze właściwe zadania. 

W razie potrzeby korzystaj z https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/index.html

# 20 Years of Games

**7. Zaczytaj do zmiennej gameInfosDF zawartość pliku ign.csv**

In [2]:
username = "maciej_wieczorek" # UWAGA! ustaw zmienną username na poprawną wartość

gameInfosDF=spark.read.option("inferSchema", "true").csv(f"ign.csv", header=True).cache()

**8. Wyświetl schemat zmiennej gameInfosDF**

In [3]:
gameInfosDF.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- score_phrase: string (nullable = true)
 |-- title: string (nullable = true)
 |-- url: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- score: double (nullable = true)
 |-- genre: string (nullable = true)
 |-- editors_choice: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- release_month: integer (nullable = true)
 |-- release_day: integer (nullable = true)



Możesz także po prostu przyglądnąć się jej kolumnom

In [4]:
gameInfosDF.columns

['_c0',
 'score_phrase',
 'title',
 'url',
 'platform',
 'score',
 'genre',
 'editors_choice',
 'release_year',
 'release_month',
 'release_day']

Zobaczmy też trzy pierwsze wiersze. Zróbmy to na kilka sposobów. 

* Na początek metoda `show()`

In [8]:
gameInfosDF.limit(3).show()

+---+------------+--------------------+--------------------+----------------+-----+----------+--------------+------------+-------------+-----------+
|_c0|score_phrase|               title|                 url|        platform|score|     genre|editors_choice|release_year|release_month|release_day|
+---+------------+--------------------+--------------------+----------------+-----+----------+--------------+------------+-------------+-----------+
|  0|     Amazing|LittleBigPlanet P...|/games/littlebigp...|PlayStation Vita|  9.0|Platformer|             Y|        2012|            9|         12|
|  1|     Amazing|LittleBigPlanet P...|/games/littlebigp...|PlayStation Vita|  9.0|Platformer|             Y|        2012|            9|         12|
|  2|       Great|Splice: Tree of Life|/games/splice/ipa...|            iPad|  8.5|    Puzzle|             N|        2012|            9|         12|
+---+------------+--------------------+--------------------+----------------+-----+----------+------------

Przetwarzane dane mogą być duże. Wyniki natomiast z reguły są znacznie mniejsze, to pozwala nam je (o ile znamy ich wielkość) przekonwertować do obiektów `pandas DataFrame` i dzięki temu przedstawić w przyjaźniejszej postaci.
* metoda `toPandas()`

In [7]:
import pandas
gameInfosDF.limit(3).toPandas()

Py4JError: An error occurred while calling o63.pandasStructHandlingMode. Trace:
py4j.Py4JException: Method pandasStructHandlingMode([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:829)



Za pomocą parametru konfiguracyjnego `spark.sql.repl.eagerEval.enabled` naszego kontekstu, również możemy 
ułatwić sobie wgląd w zawartość naszych wyników. Warto także ustawić parametr aby kontrolować liczbę pobieranych w ten sposób wierszy (tak, w razie niedoszacowania wyniku)
* parametr `spark.sql.repl.eagerEval.enabled`

In [9]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
spark.conf.set('spark.sql.repl.eagerEval.maxNumRows', 3)
gameInfosDF

Py4JError: An error occurred while calling o32.htmlString. Trace:
py4j.Py4JException: Method htmlString([class java.lang.Integer, class java.lang.Integer]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:829)



+---+------------+--------------------+--------------------+----------------+-----+----------+--------------+------------+-------------+-----------+
|_c0|score_phrase|               title|                 url|        platform|score|     genre|editors_choice|release_year|release_month|release_day|
+---+------------+--------------------+--------------------+----------------+-----+----------+--------------+------------+-------------+-----------+
|  0|     Amazing|LittleBigPlanet P...|/games/littlebigp...|PlayStation Vita|  9.0|Platformer|             Y|        2012|            9|         12|
|  1|     Amazing|LittleBigPlanet P...|/games/littlebigp...|PlayStation Vita|  9.0|Platformer|             Y|        2012|            9|         12|
|  2|       Great|Splice: Tree of Life|/games/splice/ipa...|            iPad|  8.5|    Puzzle|             N|        2012|            9|         12|
+---+------------+--------------------+--------------------+----------------+-----+----------+------------

Wykorzystuj powyższe, aby móc podglądać uzyskiwane wyniki

 
**9. Na początek coś prostego. 
Wyświetl trzy najlepiej ocenione gry wydane w roku 2016 na platformę PC.**

In [47]:
from pyspark.sql.functions import col, lit
# tu wprowadź swoje rozwiazanie
# gameInfosDF.select(col("platform") == "PC").limit(3).show("title")
gameInfosDF.where(col("release_year") == "2016").where(col("platform") == "PC").orderBy(col("score").desc()).limit(3).select("title", "score").show()
# gameInfosDF.select(col("platform") == "PC"),(col("release_year") == "2016").orderby(Odpowiedź("score").Odpowiedź()).limit(3).show("title","score")

+-----------+-----+
|      title|score|
+-----------+-----+
|  Undertale| 10.0|
|The Witness| 10.0|
|     Inside| 10.0|
+-----------+-----+



**10. Określ dla każdej oceny opisowej (score_phrase) minimalną i 
maksymalną ocenę liczbową. Wyniki posortuj
rosnąco pod względem minimalnej oceny liczbowej.**

In [52]:
from pyspark.sql.functions import *
spark.conf.set('spark.sql.repl.eagerEval.maxNumRows', 20)
# tu wprowadź swoje rozwiazanie
gameInfosDF.groupBy("score_phrase").agg(max("score"), min("score").alias("score-min")).orderBy("score-min")

Py4JError: An error occurred while calling o497.htmlString. Trace:
py4j.Py4JException: Method htmlString([class java.lang.Integer, class java.lang.Integer]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConnection.java:106)
	at java.base/java.lang.Thread.run(Thread.java:829)



+------------+----------+---------+
|score_phrase|max(score)|score-min|
+------------+----------+---------+
|    Disaster|       0.8|      0.5|
|  Unbearable|       1.9|      1.0|
|     Painful|       2.9|      2.0|
|       Awful|       3.9|      3.0|
|         Bad|       4.9|      4.0|
|    Mediocre|       5.9|      5.0|
|        Okay|       6.9|      6.0|
|        Good|       7.9|      7.0|
|       Great|       8.9|      8.0|
|     Amazing|       9.9|      9.0|
| Masterpiece|      10.0|     10.0|
+------------+----------+---------+

**11. To może coś trudniejszego. Wyznacz liczbę oraz średnią ocenę gier wydawanych w poszczególnych latach
począwszy od roku 2000 na poszczególne platformy. Nie analizuj wszystkich platform – ogranicz je tylko do
tych, dla których liczba wszystkich recenzji gier biorąc pod uwagę wszystkie lata przekroczyła 500.**

*Uwaga: Klasycznie odwołalibyśmy się do źródłowego zboru danych dwa razy. Raz aby wyznaczyć popularne platformy, a następnie aby wyznaczyć ostateczny wynik. 
Korzystając z funkcji analitycznych możesz to zadanie rozwiązać sięgając do źródłowych danych tylko raz.*

**Rozwiąż to zadanie na dwa sposoby:**

a. Za pomocą DataFrame API


In [ ]:
from pyspark.sql.window import Window
# tu wprowadź swoje rozwiazanie


b. Za pomocą SQL (po zarejestrowaniu źródeł danych jako tymczasowych perspektyw).

In [ ]:
gameInfosDF.createOrReplaceTempView("gameinfos")
# tu wprowadź swoje rozwiazanie


**12. Jeśli masz swoją ulubioną serię gier (https://pl.wikipedia.org/wiki/Kategoria:Serie_gier_komputerowych)
zobacz jakie średnie oceny zdobyły poszczególne pozycje z tej serii. Wyniki posortuj chronologicznie.**

In [ ]:
# tu wprowadź swoje rozwiazanie


**13. (opcjonalne) Porównaj ze sobą gry wchodzące w skład wybranych serii gier wchodzących w skład 20
najlepszych serii wg Guinessa (lista z 2010 roku). W związku z tym, że gry nie są wydawane co roku, pogrupuj
dane w przedziały o długości 5 lat.**

In [ ]:
# tu wprowadź swoje rozwiazanie


In [ ]:
# brudnopis

 
# MondialDB – DataFrames

**14. Na początku do zmiennych `citiesDF`, `countriesDF` załaduj odpowiednio dane z plików
`mondial.cities.json`, `mondial.countries.json`**

In [ ]:
citiesDF = spark.read.json(f"/user/{username}/mondial.cities.json").cache()
countriesDF = spark.read.json(f"/user/{username}/mondial.countries.json").cache()

**15. Zapoznaj się z ich strukturą. Zwróć uwagę na występujące typy array.**

In [ ]:
citiesDF.printSchema()
countriesDF.printSchema()

**16. Zanim zaczniesz realizować zadania, zapoznaj się ze funkcją `explode`, która nadaje się świetnie do pracy z tablicami i ich rozpłaszczania.**

**Przykładowe zapytanie:**

In [ ]:
countriesDF.where("name = 'Poland'").\
            select(col("name"), explode(col("population")).alias("pop_in_years"))

Zwróć uwagę także na inne funkcje z tej rodziny jak: `explode_outer`, `posexplode`, `posexplode_outer`
https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html


Wszystkie zadania wykonaj korzystając *DataFrame API*. Nie korzystaj z SQL.

**17. Oblicz sumę ludności wszystkich Państw na rok 2010. 
W sytuacji gdy w danym kraju nie przeprowadzono
badania w roku 2010 wykorzystaj najnowsze z badań wcześniejszych.**

In [ ]:
from pyspark.sql.window import Window
# tu wprowadź swoje rozwiazanie


**18. Było ciężko? Nie wierzę.**

**Teraz już będzie z górki. Podaj nazwy i gęstość zaludnienia trzech krajów o największej gęstości zaludnienia w roku 2010.**

In [ ]:
# tu wprowadź swoje rozwiazanie



**19. Podaj trzy kraje o największym procencie ludności żyjącym w miastach powyżej 50 000 mieszkańców w roku
2010.**

In [ ]:
# tu wprowadź swoje rozwiazanie


No cóż, dane dotyczące ludności w miastach są zapewne nowsze niż z 2010 roku.
Na marginesie, zarówno Melilla jak i Ceuta to hiszpańskie miasta, afrykańskie eksklawy położone na terytorium
Maroka. Oba liczą ponad 70 tyś mieszkańców i oba posiadają autonomię (uzyskaną jednocześnie w marcu 1995 roku)
dlatego znalazły się w naszym zestawieniu.
A co to takiego eksklawy i czy enklawa jest tym samym, to już możesz przeczytać samodzielnie np. tu:
https://pl.wikipedia.org/wiki/Eksklawa

In [ ]:
# brudnopis